# Задача и данные
Требуется разработать модель, которая будет способна различать заголовки реальных и выдуманных новостей.
Наши данные - заголовки новостей, лейбл - является ли новость фейком <br>
Задача: по заголовку определить является ли новость фейком <br>

Первая модель: поиск с google search (лучшее по качеству) <br>
Вторая модель: трансформеры и ELMO

In [15]:
!pip install beautifulsoup4
!pip install google

In [3]:
import pandas as pd

train_df = pd.read_csv('./dataset/train.tsv', delimiter='\t')
train_df.head(5)

,title,is_fake
0,Москвичу Владимиру Клутину пришёл счёт за вмеш...,1
1,Агент Кокорина назвал езду по встречке житейск...,0
2,Госдума рассмотрит возможность введения секрет...,1
3,ФАС заблокировала поставку скоростных трамваев...,0
4,Против Навального завели дело о недоносительст...,1


Возьмем новость фейк и не фейк и заметим, что по этим запросам находятся страницы в поиске. Взяв выборку примерно из 10 фейковых новостей можно увидеть что они взяты с сайта фейковых новостей panorama.pub
Предполагаю, что лучшая модель для решения задачи не включает в себя статистических языковых моделей, а делается простым поиском

из интересных находок: 
* (xe-xe-xe) новость <br> Заголовок: Россияне обхитрили рост цен <br> URL: https://lenta.ru/news/2018/04/17/xe_xe_xe/ 

## Поиск гугл
Гугл дает ограничение по запросам, нужно выставлять искусственную задержку между запросами <br>
На найденных 53 новостях точность F1 мера показала 1.0 <br>
Этого достаточно чтобы утверждать, что эта модель будет или идеальной, или близка к идеальной <br>
т.к. один из критериев оценки - F1 мера точности, я включу эту модель в финальное решение

In [23]:
try:
    from googlesearch import search
except ImportError:
    print("No module named 'google' found")
from time import sleep

titles = train_df['title'].values
labels = []
for title in titles:
    query = title
    # print(query)
    for j in search(query, tld="co.in", num=1, stop=10, pause=2):
        if 'panorama' in j:
            labels.append(1)
        else:
            labels.append(0)
        # print(j, labels[-1])
        time.sleep(1)
        break

HTTPError: HTTP Error 429: Too Many Requests

In [26]:
len(labels)

53

In [25]:
from sklearn.metrics import f1_score
y_true = train_df['is_fake'].values[:len(labels)]
y_pred = labels
f1_score(y_true, y_pred, average='macro')

1.0

## Статистические языковые модели
Хорошо, мы смогли понять откуда данные и получить отличную F1 метрику, теперь интересная часть <br>
Смогут ли современные языковые модели по заголовку понять, фейковая ли новость? <br>
Посмотрев на данные своими глазами и попробовав решить эту задачу без помощи автоматики, могу сказать что я затрудняюсь сказать по заголовку новости фейк это или нет. <br>

**В этой задаче важно иметь subword токенизацию, так как именованные сущности встречаются почти в каждом заголовке**<br>
Считаю, что стоит попробовать такие подходы как: <br>
1. Векторизация TF-IDF (или любая другая токен-векторизовалка) + MLP (самый слабый из трех, потому что скорее всего не хватит Term'ов для редких аббревиатур, условно "ФННБ" встретится один раз и будет мало веса добавлять к вектору)
2. ELMO (хороший tradeoff качество/скорость)
3. BERT (лучшее по качеству) 
4. sGPT (один из новых подходов, интересно попробовать) <br>

Считаю, что не стоит включать word2vec в исследование, т.к. корпус включает в себя примеры со многим кол-вом именованных существительных и все неизвестные имена и названия при токенизации w2v будут отмечены как *UNK*, а что в решении того, фейк новость или нет, это не поможет <br>

### Data split

In [37]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(
    train_df['title'].tolist(), train_df['is_fake'].tolist(), test_size=0.2, random_state=42)

### TF-IDF + classifier

In [40]:
from sklearn.linear_model import SGDClassifier
from sklearn.pipeline import Pipeline

text_clf = Pipeline([
    ('vect', TfidfVectorizer()),
    ('clf', SGDClassifier(loss='hinge', penalty='l2',
                          alpha=1e-3, random_state=42,
                          max_iter=5, tol=None)),
])

text_clf.fit(X_train, y_train)

predicted = text_clf.predict(X_test)

In [41]:
from sklearn.metrics import f1_score
y_true = y_test
y_pred = predicted
f1_score(y_true, y_pred, average='macro')

0.7819391183357655

### ELMO (WMT news)

In [8]:
# !pip install scikit-learn
# import sys
# !{sys.executable} -m pip install --user deeppavlov
# !python -m deeppavlov install elmo_ru-news

In [9]:
# import tensorflow as tf
# import tensorflow_hub as hub

# elmo = hub.Module("http://files.deeppavlov.ai/deeppavlov_data/elmo_ru-news_wmt11-16_1.5M_steps.tar.gz",
# trainable=True)
# sess = tf.Session()
# sess.run(tf.global_variables_initializer())
# embeddings = elmo(["это предложение", "word"], signature="default", as_dict=True)["elmo"]
# sess.run(embeddings)

### BERT

In [11]:
!pip install -r bertRequirements.txt

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 804.1/804.1 MB 2.8 MB/s eta 0:00:0000:0100:02
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 23.7 MB/s eta 0:00:00a 0:00:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.1/2.1 MB 21.8 MB/s eta 0:00:00a 0:00:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.3/9.3 MB 20.8 MB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 14.9/14.9 MB 20.9 MB/s eta 0:00:0000:0100:01
ERROR: Could not find a version that satisfies the requirement scikit-learn==0.22.2 (from versions: 0.9, 0.10, 0.11, 0.12, 0.12.1, 0.13, 0.13.1, 0.14, 0.14.1, 0.15.0b1, 0.15.0b2, 0.15.0, 0.15.1, 0.15.2, 0.16b1, 0.16.0, 0.16.1, 0.17b1, 0.17, 0.17.1, 0.18, 0.18.1, 0.18.2, 0.19b2, 0.19.0, 0.19.1, 0.19.2, 0.20rc1, 0.20.0, 0.20.1, 0.20.2, 0.20.3, 0.20.4, 0.21rc2, 0.21.1, 0.21.2, 0.21.3, 0.22rc2.post1, 0.22rc3, 0.22, 0.22.1, 0.22.2.post1, 0.23.0rc1, 0.23.0, 0.23.1, 0.23.2, 0.24.dev0, 0.24.0rc1, 0.24.0, 0.24.1, 0.24.2, 1.0rc1, 1.0rc2, 1.0, 1.0.1, 1.0.2)


In [13]:
from bert_dataset import CustomDataset
from bert_classifier import BertClassifier

TypeError: Unable to convert function return value to a Python type! The signature was
	() -> handle

In [ ]:
classifier = BertClassifier(
        model_path='cointegrated/rubert-tiny',
        tokenizer_path='cointegrated/rubert-tiny',
        n_classes=2,
        epochs=2,
        model_save_path='/content/bert.pt'
)

In [ ]:
classifier.preparation(
        X_train=list(train_data['text']),
        y_train=list(train_data['label']),
        X_valid=list(valid_data['text']),
        y_valid=list(valid_data['label'])
    )

In [ ]:
classifier.train()

In [ ]:
texts = list(test_data['text'])
labels = list(test_data['label'])

predictions = [classifier.predict(t) for t in texts]

In [ ]:
from sklearn.metrics import f1_score
y_true = y_test
y_pred = predicted
f1_score(y_true, y_pred, average='macro')